# Project 2: Corona/Weather Data Science - FYP 2021
## How weather influences the spread of the pandemic 
---
### Group 9: Aidan Stocks, Christian Margo Hansen, Jonas-Mika Senghaas, Malthe Pabst, Rasmus Bondo Hansen
Submission: 19.03.2021 / Last Modified: 05.03.2021

---

This notebook contains the step-by-step data science process performed on the `IBM Wheather Data` from 2020 and the official `Corona Statistics` in 2020. 
The goal was to inform the authorities of **Germany** (Group 9 Focus) about the development of the pandemic in 2020 and find possible associations to environmental conditions. 

The raw datasets were given by the course mananger. *Link source*

> *Contact for technical difficulties or questions related to code: Jonas-Mika Senghaas (jsen@itu.dk)*

### Introduction
The COVID-19 pandemic, also known as the coronavirus pandemic, is an ongoing pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). It was first identified in December 2019 in Wuhan, China. The World Health Organization declared the outbreak a Public Health Emergency of International Concern in January 2020 and a pandemic in March 2020. As of 5 March 2021, more than 115 million cases have been confirmed, with more than 2.56 million deaths attributed to COVID-19, making it one of the deadliest pandemics in history. (Source: [Wikipedia](https://en.wikipedia.org/wiki/COVID-19_pandemic)).

The goal of this project is to analyse the development of the pandemic in Germany for the year 2020, while paying special attention to environmental factors that may relate to higher chances of infection and therefore a faster spread of the pandemic.

## Running this Notebook
---
This notebook contains all code to reproduce the findings of the project as can be seen on the [GitHub page](https://github.com/jonas-mika/fyp2021p02g09) of this project. In order to read in the data correctly, the global paths configured in the section `Constants` need to be correct. The following file structure - as prepared in the `submission.zip` - was followed throughout the project and is recommended to use (alternatively the paths in the section `Constants` can be adjusted):

```
submission
|   github_link.webloc
│   project_report.pdf
│   gitlog.txt    
│
└───data
│   │
│   | raw
│   | │   
│   | │   
│   | │   
|   |
|   | 
|  
│   
└───notebooks
    |   project2.ipynb (current location)
    |   project2
        |   __init__.py
        |   visualisations.py
        |   ...
    
```

# Required Libraries
---
Throughout the project, we will use a range of both built-in and external Python Libraries. This notebook will only run if all libraries and modules are correctly installed on your local machines. 
To install missing packages use `pip install <package_name>` (PIP (Python Package Index) is the central package management system, read more [here](https://pypi.org/project/pip/)). 

In case you desire further information about the used packages, click the following links to find detailled documentations:
- [Pandas Homepage](https://pandas.pydata.org/)
- [Numpy Homepage](https://numpy.org/)
- [Matplotlib Homepage](https://matplotlib.org/stable/index.html)
- [Seaborn Homepage](https://seaborn.pydata.org/)
- [Folium Documentation](https://python-visualization.github.io/folium/)
- [Scipy Homepage](https://www.scipy.org/)
- [Textwrap Documentation](https://docs.python.org/3/library/textwrap.html)

In [ ]:
import pandas as pd                                    # provides major datastructure pd.DataFrame() to store the datasets
import numpy as np                                     # used for numerical calculations and fast array manipulations
import matplotlib                                      # visualisation of data
import matplotlib.pyplot as plt                        # visualisation of data
import matplotlib.dates as mdates
import datetime as dt
import re
import seaborn as sns                                  # plotting categorical vs. numerical variables
import folium                                          # spatial visualisation
from folium.plugins import HeatMap, MarkerCluster      # spatial visualisation
from scipy.stats import chi2_contingency               # chi2 statistical association test
import json                                            # data transfer to json format
import os                                              # automates saving of export files (figures, summaries, ...)
import random                                          # randomness in coloring of plots
import textwrap                                        # prettify axis labelling

## Own Package
---
Since this project makes heavy use of functions to achieve maximal efficiency, all functions are stored externally in the package structure `project1'. The following imports are necessary for this notebook to run properly.

In [ ]:
from project2.processing import check_columns_for_missing_values
from project2.numerical_summary import get_uniques_and_counts, get_fivenumsummary, compute_numerical_summary
from project2.visualisations import initialise_summary, barplot, histogram, boxplot, categorical_scatterplot, categorical_association_test
from project2.spatial_visualisation import plot_marker, random_color, map_accidents
from project2.save import save_csv, save_json, save_figure, save_dict, save_map, save_all_single_variable_analysis, save_all_categorical_scatters, save_all_categorical_associations

**REMARK**: All function used in this project are well documented in their `Docstring`. To display the docstring and get an short summary of the function and the specifications of the input argument (including data tupe and small explanation) as well as their return value, type **`?<function_name>`** in Juptyer (*see example*)

# Constants
---
To enhance the readibilty, as well as to decrease the maintenance effort, it is useful for bigger projects to define contants that need to be accessed globally throughout the whole notebook in advance. 
The following cell contains all of those global constants. By convention, we write them in caps (https://www.python.org/dev/peps/pep-0008/#constants)

In [ ]:
# path lookup dictionary to store the relative paths from the directory containing the jupyter notebooks to important directories in the project
PATH = {}

PATH['data'] = {}
PATH['data']['raw'] = "../data/raw/"
PATH['data']['interim'] = "../data/interim/"
PATH['data']['processed'] = "../data/processed/"
PATH['data']['external'] = "../data/external/"

PATH['corona'] = 'corona/'
PATH['metadata'] = 'metadata/'
PATH['shapefiles'] = 'shapefiles/'
PATH['weather'] = 'weather/'

PATH["references"] = "../data/references/"

PATH['reports'] = "../reports/"

region_codes = []

# filename lookup dictionary storing the most relevant filenames
FILENAME = {}
FILENAME['weather'] = 'weather.csv'
FILENAME['corona'] = 'de_corona.csv'
FILENAME['metadata'] = 'de_metadata.json'
FILENAME['shapefiles'] = 'de.geojson'

# defining three dictionaries to store data. each dictionary will reference a pandas dataframe to a string-representation as a key.
DATA_RAW = {}
DATA_GERMANY = {}
DATA_REGIONS = {}
DATA_EXTERNAL = {}

for region_code in region_codes:
    DATA_REGIONS[region_code] = {}

REGION_LOOKUP = {}

# automising all plots requires a lot of additional information (ie. what plot-type to use on the different variables, whether or not we need a fivenumber-summary, etc.). this information is stored in the summary dictionary
SUMMARY = {}

for region in zip(['germany'], region_codes):
    SUMMARY[region] = {}
    
NAMES = {}
NAMES['datasets'] = ['weather', 'corona']
NAMES['jsons'] = ['metadata', 'shapefiles']

# Loading, Inspection and Processing of Datasets (TASK 0)
---

## Loading in the Datasets
---
The data analysis revolves around a handful of datasets from different resources: 
> *CSV*: Corona (DE) -  Contains the `Number of new Infections (per Day)` and `Number of new Casualties (per Day)` for (almost) each day of 2020 and each region

> *CSV*: Weather - Contains information about several indicator of weather conditions for each region in Germany, Denmark, Sweden and the Netherlands for each day during the period 13.02.2020 - 14.11.2020

> *JSON*: Metadata (DE) - Contains more information about the different regions in Germany

> *GEOJSON*: Geojson (DE) - Holds the `geojson` data for the different regions in Germany

We conveniently load in the `csv` datasets into individual Pandas `DataFrames` using the built-in pandas method `pd.read_csv()` ([Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)). We store those in our `DATA_RAW` dictionary in the corresponding keys.

For the `json` and `geojson` files, we use Pythons built-in library `json`.

### a. Datasets (Weather/ Covid)

In [ ]:
# load in weather and corona data using pandas into the predefined dictionary 'DATA_RAW' where the key corresponds to the internal dataset name
for dataset in NAMES['datasets']:
    DATA_RAW[dataset] = pd.read_csv(PATH['data']['raw'] + PATH[dataset] + FILENAME[dataset], sep = '\t')

In [ ]:
additional_weather = pd.read_csv(PATH['data']['raw'] + PATH['weather'] + 'weather2.csv', sep= '\t')
DATA_RAW['weather'] = pd.concat([DATA_RAW['weather'], additional_weather])

### b. JSONS
---

In [ ]:
# load in metadata using json library
for JSON in NAMES['jsons']:
    with open(PATH['data']['raw'] + PATH[JSON] + FILENAME[JSON]) as infile:
        DATA_RAW[JSON] = json.load(infile)

In [ ]:
REGION_LOOKUP['iso'] = {DATA_RAW['metadata']['country_metadata'][i]['iso3166-2_code']: DATA_RAW['metadata']['country_metadata'][i]['iso3166-2_name_en'] 
                        for i in range(len(DATA_RAW['metadata']['country_metadata']))} # key: iso, value: region name
REGION_LOOKUP['region'] = {DATA_RAW['metadata']['country_metadata'][i]['iso3166-2_name_en']: DATA_RAW['metadata']['country_metadata'][i]['iso3166-2_code']                            for i in range(len(DATA_RAW['metadata']['country_metadata']))} # key: region name, value: iso

In [ ]:
region_codes = [DATA_RAW['metadata']['country_metadata'][i]['iso3166-2_code'] for i in range(len(DATA_RAW['metadata']['country_metadata']))]

In [ ]:
# at other position
for region_code in region_codes:
    PATH[region_code] = region_code + '/'

## Inspection of Datasets
---
We can now have a look at our two dataset to get a good first impression for what kind of data we are dealing with. We start by reporting the number of records and fields/  variables in each of the datasets by using the `shape` property of the Pandas `DataFrame`. 
We then continue to have an actual look into the data. Similiar to the `head` command in terminal, we can use the method `head()` onto our `DataFrames`, which outputs a nice inline representation of the first five data records of the dataset.

### Size (Number of Records and Fields)

In [ ]:
for dataset in NAMES['datasets']:
    print(f"{dataset.capitalize()}: {DATA_RAW[dataset].shape}")

### Peek into Datasets (Describing Attributes)

In [ ]:
DATA_RAW['weather'].head()

We see, that the main dataset `weather` stores weather information for the four countries Germany (DE), Denmark (DK), Sweden (SE) and the Netherlands (NL) . It consist of 14.904 rows, corresponding to reports of weather conditions on a specified day in a specified region in one of the countries. The following variables appear in the dataset:

> `DATE` (YYYY-MM-DD): The day of the weather reports (*Time Attribute*)

> `ISO 3166-2`: [ISO 2 Code]() for the region, in which the weather report is (*Geographic Attribute*)

> `Relative Humidity Surface`: (*Numerical Attribute*)

> `Solar Radiation`: (*Numerical Attribute*)

> `Surface Pressure`: (*Numerical Attribute*)

> `Temperature Above Ground`: (*Numerical Attribute*)

> `Total Precipiation`: (*Numerical Attribute*)

> `UV Index`: (*Numerical Attribute*)

> `Wind Speed`: (*Numerical Attribute*)

In [ ]:
DATA_RAW['corona'].head()

We see, that the dataset `corona` holds information about newly infected cases and new deaths per day per region in Germany for most of the days in 2020. It consist of 5602 rows, corresponding to 5.602 reported days. 

> `DATE` (YYYY-MM-DD): The day of the reports of new infections and deaths per region

> `REGION CODE`: Region in Germany (*Geographic Attribute*)

> `CONFIRMED ADDITION`: The number of newly confirmed infections in the region on the specified day (*Numerical Attributes*)

> `DECEASED ADDITION`: The number of newly confirmed deaths in the region on the specified day (*Numerical Attributes*)

### Summary
...

## Initial Sanity Check
---
Before continuing with the data analysis, we want to make sure that the datasets are clean. There are a plentiful of methods to check this. In the following, we will stick to the following three:
>(a) **Missing Values** (*Are there missing values? If yes, how many and in which columns?*)

>(b) 

### Missing Values
We check in all columns in all datasets the number of missing values (encoded as `' '` (*empty strings*)) to see if there are missing values (empty string) to get a feeling on which columns we need to further do processing.

In [ ]:
for dataset in NAMES['datasets']:
    print(dataset.capitalize())
    check_columns_for_missing_values(DATA_RAW[dataset])
    print('\n')

## Filtered 
---


In [ ]:
for JSON in NAMES['jsons']:
    DATA_GERMANY[JSON] = DATA_RAW[JSON]

In [ ]:
DATA_GERMANY['corona'] = {}
DATA_GERMANY['corona']['all'] = DATA_RAW['corona']

In [ ]:
DATA_GERMANY['weather'] = {}
DATA_GERMANY['weather']['all'] = DATA_RAW['weather'][DATA_RAW['weather']['iso3166-2'].isin(region_codes)]

In [ ]:
for region in region_codes:
    DATA_GERMANY['weather'][REGION_LOOKUP['iso'][region].lower()] = DATA_GERMANY['weather']['all'][DATA_GERMANY['weather']['all']['iso3166-2'] == region]

In [ ]:
for region in region_codes:
    DATA_GERMANY['corona'][REGION_LOOKUP['iso'][region].lower()] = DATA_GERMANY['corona']['all'][DATA_GERMANY['corona']['all']['region_code'] == REGION_LOOKUP['iso'][region]]

### Inspection
---
With our newly filtered datasets, we have reduced the number of records about accidents, casualties and vehicles in the datasets `accidents`, `casualties` and `vehicles`, respectively. To see, with what kind of data we are dealing now, it makes to recompute the size of each of the datasets. However, our analysis of the column attributes still holds, since we have only filtered across `axis=0` (the rows), while we left the attributes untouched. 

Again, we report the number of records and fields/ variables in each of the datasets by using the `shape` property of the Pandas `DataFrame`. 

In [ ]:
for dataset in NAMES['datasets']:
    print(f"{'-'*5}{dataset.capitalize()}{'-'*5}")
    for key in DATA_GERMANY[dataset].keys():
        print(f"{key.title()}: {DATA_GERMANY[dataset][key].shape}")

### Saving Filtered Data
---
After having filtered the raw datasets into a filted one, that only contains records about accidents located in Leeds, we want to save this interims version of the data as a `csv` into an interim folder. 

In [ ]:
for dataset in NAMES['datasets']:
    for key in DATA_GERMANY[dataset].keys():
        save_csv(DATA_GERMANY[dataset][key], path=PATH['data']['interim'] + PATH[dataset], filename=f"{dataset}_{key}", index=False) 

In [ ]:
for JSON in NAMES['jsons']:
    save_json(DATA_GERMANY[JSON], path=PATH['data']['interim'] + PATH[JSON], filename=FILENAME[JSON])

## Process Data
---
Luckily, the data is already quite clean. 

* Here we need to process the data *

In [ ]:
for dataframe in DATA_GERMANY['weather'].keys():
    DATA_GERMANY['weather'][dataframe].rename(columns={'date': 'Date', 'iso3166-2': 'ISO3166-2','RelativeHumiditySurface': 'Relative Humidity Surface', 'SolarRadiation': 'Solar Radiation', 'Surfacepressure': 'Surface Pressure', 'TemperatureAboveGround': 'Temperature Above Ground', 'Totalprecipitation': 'Total Precipiation', 'UVIndex': 'UV-Index', 'WindSpeed': 'Wind Speed'}, inplace=True)

## Export Processed Datasets
--- 
Finally, we export the processed datasets into a new subfolder. From now on, all Jupyter Notebooks will work with those processed datasets.

In [ ]:
for dataset in NAMES['datasets']:
    for key in DATA_GERMANY[dataset].keys():
        save_csv(DATA_GERMANY[dataset][key], path=PATH['data']['processed'] + PATH[dataset], filename=f"{dataset}_{key}", index=False) 

# Single Variable Analysis (TASK 1)
---
We have obtained filtered and processed datasets from TASK 0. We will now turn our focus to analysing each attribute in each of the datasets both in a numerical summary (ie. through the number of uniques, counts of uniques or a five-number summary, where it makes sense) and visual representation. Depending on the type of the variable, there arouse different preferred ways of visual repsentation, to get a visual feeling for the data we are dealing with. 
> `Barplot` (*Categorical Variables*)
>> We usually want to plot categorical variables in a bar plot, where the x-axis is labelled with the unique values measured in the specific column and the y-axis represents the number of occurences of each unique value. An example of a bar plot in this project is ie. the Number of Accidents per Week Day.

> `Histogram` (*Numerical Variables*)
>> Most numerical variables (especially those with a wide range), are plotted in a histogram (which accounts for the fact that numerical variables are continuous). An example of a histogram is ie. the Age Distribution of Casualties involved in an Accident.

> `None` (*Others*)
>> For some variables it doesn't make sense to plot them, ie. geographical (`Longitude`, ...) or relatinonal attributes (`Accident_Index`, ...).

## Initialise Summaries 
---
In order to automate the process of creating numerical summaries and appropriate visualisations for each plot, we create a central datastructure for each of the datasets, which we call `SUMMARY` (*see this in `Constants`). The `SUMMARY` datastructure is a rich datastructure defined for each dataset,which initially contains the name, label behavior (*Does the column have a Lookup?*), plotting behavior (*How do we want to plot the column?*) and information about the five-number summary (*Whether or not we want to compute a five-number summary?*). 
Later we will fill this data structure even further and use is as a basis for all our plots. 

However, the first step is to manually inspect all columns and decide on how we want to deal with them. We hardcode these observation into three individually dictionaries defined for each dataset.

In [ ]:
# dictinary that for each column in each dataset specifies the type of plot to use
PLOTTING = {}
PLOTTING['weather'] = [None, None, 'line', 'line', 'line', 'line', 'line', 'line', 'line']
PLOTTING['corona'] = [None, None, 'bar', 'bar']

In [ ]:
for dataset, start_at in zip(TABLENAMES, [2, 37, 22]): # start_at corresponds to sheet indexes at which it switches to new dataset
    initialise_summary(
        data = DATA_LEEDS[dataset], 
        lookup = VARIABLE_LOOKUP, 
        dataset_name = dataset, 
        key = dataset, 
        summary = SUMMARY, 
        labels = LABELS[dataset], 
        plotting = PLOTTING[dataset], 
        fivenum = FIVENUM[dataset], 
        start_at=start_at)

## Numerical Analysis
---
We have defined our central datastructure `SUMMARY` that currenlty contains information on how to process each column in each dataset. Given the column index as a key it stores...
> ...the name of the attribute at `['Name']`

> ...the way we would like to plot the attribute at `['Plot']`

> ...whether or not we need a five-number summary at `['Summary']`

> ...the mapping from the variable lookup (if there exist one) at `['Map']`

However, the dictionary does not yet contain any data or actual summaries. We will change this in the following section. With the help of the small helper-function `get_uniques_and_counts` and `get_fivenumsummary`, we can easily iterate over the summaries of each dataset in `compute_numerical_summary` and depending on the specified properties of each column, make computations. 

In [ ]:
for dataset in TABLENAMES:
    compute_numerical_summary(SUMMARY[dataset], DATA_LEEDS[dataset])

Great! For each dataset, we have computed the relevant data in each column. Depending on the properties, the following properties have been added...

> ...the number of uniques values at `['No_Uniques']` for all columns (independent of their type)

> ...a five-number summary at `['Five_Number_Summary']` if it is meaningful to compute a five-number summary for the column (ie. `['Summary']`==True)

> ...a dictionary of uniques and counts at `['Uniques']` if the column should be visualised as a barplot (ie. `['Plot']`=='bar')

> ...a data property storing all values in the column at `['Data']` if the column should be visualised as a histogram or barplot (ie. `['Plot']`=='hist')

### Saving Numerical Reports
We have now computed all relevant data necessary to plot our data. Before doing so, we want to save the numerical summaries using our function `save_numerical_report`. The function takes in the `SUMMARY` datastructures and saves it either as a `csv` or `json` into the specified path for further inspection and external reports. The ouput shows into which directory the files were saved.

In [ ]:
for dataset in TABLENAMES:
    save_numerical_report(SUMMARY[dataset], path=PATH['reports']['leeds'] + PATH[dataset] + 'numerical_summary/', filename=dataset,save_to='csv')
    save_numerical_report(SUMMARY[dataset], path=PATH['reports']['leeds'] + PATH[dataset] + 'numerical_summary/', filename=dataset,save_to='json')

## Visualisation
---
An essential step of each data exploration is to visualise our data. Only this makes the abstract numbers and characters meaningful.

We are therefore visualising the results of the single variable analysis in this section using the data structure `SUMMARY` we have step-by-step developed throughout the project. Depending on the properties of the attribute to plot, there arise three types of visualisatin:
> `Barplot`: The standard way of representing `categorical variables` using rectangular bars with heights and widths ([More information](https://en.wikipedia.org/wiki/Bar_chart))

> `Histogram`: A histogram is an approximate representation of the distribution of numerical data ([More information](https://en.wikipedia.org/wiki/Histogram))

> `Boxplot`: Boxplots are an important statistical tool to represent five-number summaries of numerical data visually ([More information](https://en.wikipedia.org/wiki/Box_plot))

To efficiently plot everything, we defined three unique functions `barplot`, `histogram` and `boxplot` that all take in the `SUMMARY` data structure as their main argument and 'intelligently' combine all informatation we have previously gathered to give a nice visualisation of the data.
We use the function `save_all_single_variable_analysis` that depends on `save_figure` to iterate over all columns and for each attributes computes and saves the correct plot in the correct directory. 

In [ ]:
min(DATA_GERMANY['weather']['all']['UVIndex'])

In [ ]:
def plot_weather(data, region, condition, dimensions=(16,9)):
    fig = plt.figure(figsize=(16,9))
    ax = fig.add_axes([.15,.15,.7,.7]) # [left, bottom, width, height]

    if region == 'all':
        dates = np.unique(data[region]['Date'])
        x = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in dates]
        
        y = []
        unique_dates = np.unique(data[region]['Date'])
        for unique_date in unique_dates:
            mask = data[region]['Date'] == unique_date
            masked_frame = data[region][mask][condition]
            y.append(np.mean(masked_frame))
        
        title = f"{condition} in Germany (Period: Feb 2020-Feb 2021)"
        
    else: # standard regions
        dates = data[region]['Date']
        x = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in dates] 
        y = data[region][condition]

        title = f"{condition} in {region.title()} (Period: Feb 2020-Feb 2021)"

    # plot
    ax.plot(x,y, color='darkblue')

    # labeling
    ax.set_title(title, fontweight='bold', fontsize=14)
    ax.set_xlabel("Date")
    ax.set_ylabel(f"{condition}")

    # labelling of x-axis (https://stackoverflow.com/questions/9627686/plotting-dates-on-the-x-axis-with-pythons-matplotlib)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gcf().autofmt_xdate()

    ax.set_ylim(min(data['all'][condition]), max(data['all'][condition]))

    # activate grid
    ax.grid(True)

    return fig

In [ ]:
plot_weather(
    data = DATA_GERMANY['weather'],
    region = 'baden-württemberg',
    condition = 'UV-Index'
)

In [ ]:
def plot_weather_all(data, condition, dimensions=(16,9)):
    fig = plt.figure(figsize=dimensions)
    ax = fig.add_axes([.15,.15,.7,.7]) # [left, bottom, width, height]

    for region in list(DATA_GERMANY['weather'].keys())[1:]:
        dates = data[region]['Date']
        x = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in dates] 
        y = data[region][condition]
        # plot
        ax.plot(x,y, label=region.title())

    # labeling
    ax.set_title(f"{condition} in all Regions (Feb 2020-Feb 2021)", fontweight='bold', fontsize=14)
    ax.set_xlabel("Date")
    ax.set_ylabel(f"{condition}")

    # labelling of x-axis (https://stackoverflow.com/questions/9627686/plotting-dates-on-the-x-axis-with-pythons-matplotlib)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gcf().autofmt_xdate()

    ax.set_ylim(min(data['all'][condition]), max(data['all'][condition]))

    # activate grid and legend
    ax.grid(True)
    ax.legend(loc='upper right')

    return fig

In [ ]:
plot_weather_all(
    data = DATA_GERMANY['weather'],
    condition = 'UV-Index'
)

## Saving Figures
---
For our report, we want to have nice visualisations for all our figures. To achieve this, we use the helper-function `save_figure` that depends on the property `.savefig()` of `maplotlib.pyplot.Figure` objects. We call this function on all the plots of our datasets, which automatically saves all figures into the correct directories. When executed with the path specified below, the figures are in the following directory:
> `../reports/leeds/accidents/single_variable_analysis`

> `../reports/leeds/casualties/single_variable_analysis`

> `../reports/leeds/vehicles/single_variable_analysis`

**REMARK**: Be aware that the cell takes some time to compute, since it plots and saves all figures at once.

In [ ]:
%%capture
for region in DATA_GERMANY['weather'].keys():
    for condition in list(DATA_GERMANY['weather']['all'])[2:]:
        save_figure(
            figure = plot_weather(
                data = DATA_GERMANY['weather'],
                region = region,
                condition = condition),
            path = PATH['reports'] + f"{region}/weather",
            filename = f"{condition.lower().replace(' ', '_')}",
            save_to = 'pdf'
)

## Reports of Single Variable Analysis
---
`TASK 1` specifically asks us to report the frequency of accidents...
> a. ...in different age groups

> b. ...in different times of the day, week or year

> c. ...in one other condition

The following reports (that can also be found in the automatically generated plots in `reports/leeds/`) answer these questions.

### Accident Frequency for different age groups

In [ ]:
accidents_per_age_band = barplot(SUMMARY['vehicles'][16],  keep_missing_values=False) # age band
accidents_per_age = histogram(SUMMARY['vehicles'][15], keep_missing_values=False) # age

### Accident Frequency for different times of the day, week and year

In [ ]:
accidents_per_hour_of_day = barplot(SUMMARY['accidents'][11], keep_missing_values=False) # time
accidents_per_day_of_week = barplot(SUMMARY['accidents'][10], keep_missing_values=False) # day of week
accidents_per_month_of_year = barplot(SUMMARY['accidents'][9], keep_missing_values=False) # date

### Accident Frequency for different Environmental Conditions

In [ ]:
accidents_by_light_condition = barplot(SUMMARY['accidents'][24], keep_missing_values=False)
accidents_by_weather_condition = barplot(SUMMARY['accidents'][25], keep_missing_values=False)
accidents_by_road_surface_condition = barplot(SUMMARY['accidents'][26], keep_missing_values=False)

# Associations (TASK 2)
---
An equally important, and perhaps more challenging, part of any data analysis is to find associations in a dataset. Especially when analysing *Road Safety* this is a key aspect of the analysis, as we can investigate which accident, casualty or vehicle attribute lead to more frequent or severe accidents. These insights, in turn, allow us to proactively counteract through political measures. 

Since our dataset is dominated by `categorical variables` (as seen in the `Initial Inspection of Datasets`), we mostly want to relate `categorical variables` to each other and the few `numerical variables` that exist there. The following methods of associating are used in the following section to investigate these associations:
> **`Categorical/ Numerical`**
>> For relating a categorical to a numerical variable we use so-called `categorical scatterplots`, which are a special kind of scatterplot which displays the distribution of a numerical variable for different attributes in the cateogrical variable. To plot these kind of plots we use our own function `categorical_scatterplot` that at its core depends on `sns.catplot` ([Documentation](https://seaborn.pydata.org/generated/seaborn.catplot.html))

> **`Categorical/ Categorical`**
>> Relating two categorical attributes to one another is slightly more difficult. To do this, we count the number of occurrences in all possible combinations of the two categorical variables and plot those in individual plots. To investigate the associativity between the two categorical variables, we use the [Pearson Chi Squared](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test) test. It is a statistical test applied to sets of categorical data to evaluate how likely it is that any observed difference between the sets arose by chance. It follows the general assumption, that we expect no change in the relative observed values if there is no association at all. 

## Picking Focus
---
In datasets as bis as the ones given, it is not meaningful to associate every attribute to every attribute. Instead, we want to investigate which attributes the most statistically associated to attributes that are a measure for the *Road Safety*.
Following this train of though, the most important metric is the `Accident Severity`, which we can find in the `accidents` dataset at column index `6`. It makes sense to investigate, which attributes results in the most severe accident (ie. is most associated to the `Accident Severity`).

## Linking Sub-Datasets
---
Since we also want to investigate, which `vehicle` and `casualty` attributes are most related to the `Accident Severity`, we need to prepare our data before actually associating. Namely: For each reported casualty (data record in `casualties`) and vehicle (data record in `vehicles`), we need to find the corresponding accident and its severity and map it to the specific casualty or vehicle. The following function `link_to_accidents` performs exactly this computation when called with `focus='Accident_Severity'`, as it returns a column vector of length of the number of vehicles/ casualty (in general: data.shape[0] (number of rows)), which contains the `Accident Severity` for each vehicle/ casualty. 

In [ ]:
def link_to_accidents(data, focus):
    ans = []
    for i in range(data.shape[0]):
        index = data['Accident_Index'].iloc[i]
        mask = DATA_LEEDS['accidents']['Accident_Index'] == index
        focus_column = DATA_LEEDS['accidents'][mask][focus]
        ans.append(int(focus_column))
    return np.array(ans)

In [ ]:
SEVERITY = {}
for dataset in TABLENAMES:
    SEVERITY[dataset] = link_to_accidents(DATA_LEEDS[dataset], focus='Accident_Severity')

## Saving all Associations
---

In [ ]:
%%capture
for dataset in TABLENAMES:
    save_all_categorical_scatters(data = DATA_LEEDS[dataset], summary = SUMMARY[dataset], severity = SEVERITY[dataset], path=PATH['reports']['leeds'] + PATH[dataset] + 'associations/')
    save_all_categorical_associations(data = DATA_LEEDS[dataset], severity_summary = SUMMARY['accidents'], dataset_name = dataset, summary=SUMMARY[dataset], severity= SEVERITY[dataset], path=PATH['reports']['leeds'] + PATH[dataset] + 'associations/')

## Report: Association between `Accident` and `Vehicle/ Casualty` Attribute 
---
`TASK 2` specifically asks us to show one association between either an accident and a vehicle or an accident and a casualty attribute. Since we have associated the `Accident Severity` to each numerical and categorical attribute in all datasets, we will present the most intesting association here:

**`Accident Severity / Sex of Driver`**

In [ ]:
fig, V = categorical_association_test(DATA_LEEDS['vehicles'], SUMMARY['accidents'][6], SEVERITY['vehicles'], SUMMARY['vehicles'][14], DATA_LEEDS['vehicles'].iloc[:,14]);

# Spatial Visualisation (TASK 3)
---
So far, we have completely ignored the `geographic attributes` that were recorded for each accident in `accidents`. We will change this now by plotting Leed's accidents on a map to get a good visual intuition on where the accidents happen. To do this, we use a combination of function, namely `map_accidents` that under the hood calls `plot_point`. The functions are based on `folium` - an external Python package - that was earlier introduced and makes use of leaflet.js, which is a JavaScript package used to create interactive maps. 
After export, can be explored as an `html` file in the browser or inline within Jupyter. The map will contain a heat_map (if function argument `heat_map` = `True`), and for an arbitary of column in `accidens` as an argument for `focus` will make a visual distinctions for different values of this column.

## Plot all Accidents in Leeds onto Map + Save Map Visualisation
---
Through the below cell, we save all maps generated with focuses on different attributes into the directory `../reports/leeds/maps`. We can study them in further detail by opening the `html` files in the browser.

**REMARK**: Note that the execution of the below cell may take some time, since two maps (each ~3MB) need to be rendered.

In [ ]:
%%capture
for boolean in [True, False]:
    _map = map_accidents(DATA_LEEDS['accidents'], SUMMARY['accidents'], centroid=[53.8008, -1.5491], colors=['black', 'red', 'green'], heat_map=True, marker_cluster=boolean, focus='Accident_Severity')
    save_map(_map, PATH['reports']['leeds'] + 'maps/', filename=f"Accident_Severity_Map_(marker={boolean})")

## Report: Severity Map
---
TASK 2 specifically asks us to visualise the accidents on the map of Leeds, and color-code the plotted accident by their severity. This corresponds to the map saved at the path `../reports/leeds/maps/Accident_Severity_Map(marker=True)` ([Open here](http://127.0.0.1:5500/reports/leeds/maps/6_Accident_Severity.html)). 
The below cell shows how to manually plot this map, using the `map_accidents` function. We also specify the color-coding, such that 'black'=Fatal, 'red'=Serious and 'green'=Slight.

In [ ]:
severity_map = map_accidents(DATA_LEEDS['accidents'], SUMMARY['accidents'], centroid=[53.8008, -1.5491], colors=['black', 'red', 'green'], heat_map=True, marker_cluster=False, focus='Accident_Severity')
severity_map

# Bike Safety in Leeds (TASK 4)
---
The final `TASK 4` asks us to investigate a self-chosen research question. As mentioned in the introduction to this Jupyter, our goal will be to make Leeds safer and more attractive to travel by bike. In this section we will develop...
> 1) ...why bikes are important both for a city's road safety as well as environnment

> 2) ...why Leeds is not a bike-city yet

> 3) ...what measurements should be taken (based on the data) to make Leeds safer for bikes (and thereby making biking more attractive)

https://ecf.com/news-and-events/news/data-collection-basis-better-road-safety

## 1. Influence of Bikes on Road Safety and Environment
---
Promoting the use of cycling, walking, and public transportation as a means to navigate the city is part of the city of Leeds’ plan to achieve being carbon neutral by 2030. The potential benefits of embracing, and advocating for, the widespread use of these means of transportation also extends towards increased road safety; as witnessed by the city of Oslo which reported only 1 traffic related fatality in 2019 as a result of their extensive work to make the city more bike-friendly and improve overall traffic safety. While Leeds has experienced a reduction in road casualties in recent years, the annual number of people killed or seriously injured has remained largely unchanged.


This report essentially acts to supplement the work already being done by the West Yorkshire Combined Authority through CityConnect, by investigating what factors significantly contribute to road accidents that involve cyclists, and how these might be addressed to effectively follow in Oslo’s bike tracks. The aim being to highlight any contributing factors that may be significant but unobvious, or identify accident hot-spots within Leeds that could potentially be redeveloped in the future.

## 2. Leeds' Traffic Flow 
---
Before investigating the data, it is important to get a feel for how the traffic in Leeds looks currently. We have therefore used an external dataset, that reports the annual `Road Flow` in Leeds for different types of vehicles from 2000-2019. 
The dataset was obtained from: https://roadtraffic.dft.gov.uk/local-authorities/63 on 22.02.2021 (10AM) and is located in `..data/external`

### Loading, Inspection and Processing of Dataset
---
We first load the external dataset using `pd.read_csv()` and inspect it roughly using pandas `.head()` method on `pd.DataFrame`. We get a first impression of the dataset: The dataset is already filtered for Leeds (as can be seen in `local_authority_name`) and contains the counts of different vehicle types that passed specific count points at different times. We can use this data to get a rough intuition on how the distribution of vehicles on Leeds' roads looks like:

In [ ]:
DATA_EXTERNAL['road_flow'] = pd.read_csv(PATH['data']['external'] + FILENAME['road_flow'])

In [ ]:
DATA_EXTERNAL['road_flow'].head()

For our analysis we don't need the majority of columns. We drop them here.

In [ ]:
DATA_EXTERNAL['road_flow'].drop(columns=['count_point_id', 'direction_of_travel', 'count_date', 'hour', 'region_id', 'region_name', 'local_authority_id', 'local_authority_name', 'road_name', 'road_type', 'start_junction_road_name', 'end_junction_road_name', 'easting', 'northing', 'latitude', 'longitude','link_length_km', 'link_length_miles','hgvs_2_rigid_axle', 'hgvs_3_rigid_axle', 'hgvs_3_or_4_articulated_axle', 'hgvs_5_articulated_axle', 'hgvs_6_articulated_axle'], inplace=True)

### Visualising Annual Road Flow
---
To visualise the annual `Road Flow` we need to count the occurrences of each vehicle type in different years. We do this by creating a mask for each year, and use this mask to count the occurences for each vehicle type. We save this information in a central data structure as `road_flow`. We then use standard code using `matplotlib` to create the figure that is based on the counts.

In [ ]:
road_flow = {}
for year in np.unique(DATA_EXTERNAL['road_flow']['year']):
    mask = DATA_EXTERNAL['road_flow']['year'] == year

    try: road_flow['years'].append(year)
    except: road_flow['years'] = [year]

    for i in range(1,DATA_EXTERNAL['road_flow'].shape[1]-1):
        try: road_flow[list(DATA_EXTERNAL['road_flow'])[i]].append(sum(DATA_EXTERNAL['road_flow'].iloc[:,i][mask]))
        except: road_flow[list(DATA_EXTERNAL['road_flow'])[i]] = [sum(DATA_EXTERNAL['road_flow'].iloc[:,i][mask])]

In [ ]:
# create figure and axes (with padding for better exporting)
fig = plt.figure(figsize=(16,9))
ax = fig.add_axes([.15,.15,.7,.7])    

years = road_flow['years']
labels = list(DATA_EXTERNAL['road_flow'])[1:-1]

for i in range(len(labels)):
    ax.plot(years, road_flow[labels[i]], 'o-', label=labels[i])

ax.set_xticks(years); ax.set_xticklabels(years);
ax.set_title('Annual Leeds Traffic Flow by Vehicle (2000-2019)', fontweight='bold', fontsize=16)
ax.set_xlabel('Years'); ax.set_ylabel('No. of Vehicles Counted');
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.legend();

save_figure(fig, path=PATH['reports']['external'], filename='annual_leeds_traffic_flow', save_to='pdf')

> SUMMARY: Leeds is a car-dominated city. There are hardly any bikes

## 3. Analysis
---
Due to its lack of a metro and few governmental initiatives of the government in the last few years, Leeds traffic is still highly dominated by motorised vehicles, making it dangerous for bikers. In the following section, we want to analyse specifically those accidents that involve bikes, and of each accident try to conclude patterns in the vehicle or casualty attributes of either the biker or the driver of the motorised vehicle. Some leading questions will be:
> 1. `Accident Participants`: What patterns of accident participants can we observe? 

> 2. `Single Variable Analysis`: 

>> a. Under what accident circumstances do accidents happen that involve bikes? 

>> b. What vehicle and casualty attributes can be observed for particispants of accidents that involve bikes? (Filtered SVA)

> 3. `Associations`: Which (accident/ casualty/ vehicle) attributes lead to more severe accidents involving bikes?

> 4. `Spatial Visualisation`: How can we visualise the above on an informative maps?

### Preparation of Analysis 
---
Before however, analysing the above five points, we need to prepare our data in order to match our research question. Obviously, we need to reduce our datasets to only contain information about accidents that involves bikes in Leeds. To do so, we filter in the `vehicles` dataset for `Vehicle_Type` `1`, which corresponds to Pedal Cycles. We use this filtered vehicles dataset to get all unique `Accident Indexes` bikes were involved in. 

We can then conveniently iterate over all datasets to obtain the three datasets all reduced to only contain those accident that involved at least 1 bike. We safe this in a new dictionary called `DATA_LEEDS_BIKES` with the key of the identifier of the dataset and the additional key `all`. In contrast to the previous overall Leeds analysis we need this differentiation, because we will later filter in the sub-datasets `vehicles` and `casualties` for only bikers and motorised vehicles. 

In [ ]:
bikes = DATA_LEEDS['vehicles'][DATA_LEEDS['vehicles']['Vehicle_Type'] == 1]
bike_accidents_indexes = set(bikes['Accident_Index'])

In [ ]:
for dataset in TABLENAMES:
    DATA_LEEDS_BIKES[dataset] = {}
    DATA_LEEDS_BIKES[dataset]['all'] = DATA_LEEDS[dataset][DATA_LEEDS[dataset]['Accident_Index'].isin(bike_accidents_indexes)]

For our later analysis, we also want to prepare our two sub-datasets such that each of them is filtered to only contain information on the bikers and the other only on the motorised vehicle involved in the accident. We do this in the following few cells. We want our result to be saved in `DATA_LEEDS_BIKES[<sub-dataset_name]['bikers']` and `DATA_LEEDS_BIKES[<sub-dataset_name]['motorised']` respectively. We start by masking bikers in our reduced `DATA_LEEDS_BIKES['vehicles']` dataset and motorised vehicles through the inverse of this mask (a small look into the unique participants revealed that no pedestrian were involved in the recorded bike accidents, thus all accident participants that are not bikes, is a motorised vehicle).

In [ ]:
# masking bikers and non-bikers
bikers = DATA_LEEDS_BIKES['vehicles']['all']['Vehicle_Type'] == 1 # 
motorised_vehicles = ~bikers

DATA_LEEDS_BIKES['vehicles']['bikers'] = DATA_LEEDS_BIKES['vehicles']['all'][bikers]
DATA_LEEDS_BIKES['vehicles']['motorised'] = DATA_LEEDS_BIKES['vehicles']['all'][motorised_vehicles]

Now, however we also need to separate the involved casualties of accident involving bikes into bikers and drivers of motorised vehicles. To do this, we create a mask using `link_vehicles_to_casualties`, which iterates over all `casualties` to get their accident indx and casualty reference. We use this information to mask in `DATA_LEEDS_BIKES['vehicles']['bikers']` for this specific accident and append `True` if the casualty references match, otherwise `False`. This will create a `boolean array` that we can use to filter both for biker and motorised vehicle driver casualties (using its inverse).

In [ ]:
def link_vehicles_to_casualties():
    ans = []
    for i in range(DATA_LEEDS_BIKES['casualties']['all'].shape[0]): # rows
        index = DATA_LEEDS_BIKES['casualties']['all']['Accident_Index'].iloc[i] 
        cas_ref = DATA_LEEDS_BIKES['casualties']['all']['Vehicle_Reference'].iloc[i]

        mask = DATA_LEEDS_BIKES['vehicles']['bikers']['Accident_Index'] == index
        vehicles = DATA_LEEDS_BIKES['vehicles']['bikers'][mask]
        for j in range(vehicles.shape[0]):
            if vehicles['Vehicle_Reference'].iloc[j] == cas_ref:
                ans.append(True)
            else: ans.append(False)
    return np.array(ans)

In [ ]:
DATA_LEEDS_BIKES['casualties']['bikers'] = DATA_LEEDS_BIKES['casualties']['all'][link_vehicles_to_casualties()]
DATA_LEEDS_BIKES['casualties']['motorised'] = DATA_LEEDS_BIKES['casualties']['all'][~link_vehicles_to_casualties()]

### Inspection of Datasets
---

In [ ]:
for dataset in TABLENAMES:
    print('-'*35)
    for sub_dataset in ['all', 'bikers', 'motorised']:
        try: print(f"{dataset.capitalize()} ({sub_dataset[:1].capitalize()})\t\t{DATA_LEEDS_BIKES[dataset][sub_dataset].shape}")
        except: None

### Accident Vehicle Pattern
---

In [ ]:
combinations = {}
for index in DATA_LEEDS_BIKES['accidents']['all']['Accident_Index']:
    mask = DATA_LEEDS_BIKES['vehicles']['all']['Accident_Index'] == index
    vehicles = DATA_LEEDS_BIKES['vehicles']['all'][mask]['Vehicle_Type']
    combination = sorted(tuple(vehicles))

    try: combinations[str(combination)] += 1
    except: combinations[str(combination)] = 1

In [ ]:
# create figure and axes (with padding for better exporting)
fig = plt.figure(figsize=(16,9))
ax = fig.add_axes([.15,.15,.7,.7]) # [left, bottom, width, height]

# plot 
x, y = list(combinations.keys()), list(combinations.values())
ax.barh(x, y, align='center')
ax.set_title('Accident Pattern for Bike Accidents', fontweight='bold')
ax.set_ylabel('Combinations of Accident Participants'); ax.set_xlabel('Number of Accidents (2019)')

# insert counts and percentages as text next to the corresponding bars
for x_cord, y_cord in zip(x,y):
    ax.text(y_cord, x_cord, f'{y_cord} ({str(100*round(y_cord/sum(y), 3))[:5]}%)' , color='black')

save_figure(fig, path=PATH['reports']['bikes'] + 'all/', filename='participants_pattern')

### Single-Variable Analysis
---
We have sucessfullly constrainted our datasets for bike accidents in Leeds. In a total of 238 accidents in 2019, 480 vehicles (out of which 238 were bikes and 242 were motorised vehicles) were involved in the accidents. 236 out of 240 total reported casualties were bikers, while only 4 were drivers of motorised vehicles. With these reduced datasets, we want to further investigate into the circumstances of the accidents, as well as patterns in the casualties and vehicles of both the bikers and drivers of motorised vehicles. 

We start our single-variable analysis by developing another summary datastructure that we need for plotting efficiently. We will call this datastructure `SUMMARY_BIKES` and it will be structured in the same way as `DATA_LEEDS_BIKES` is, namely with the three sub-dictionaries `all`, `bikes` and `motorised`.

#### a. Overall Bike-Accidents

---

In [ ]:
SUMMARY_BIKES = {}

In [ ]:
for dataset in TABLENAMES:
    SUMMARY_BIKES[dataset] = {}

In [ ]:
for dataset, start_at in zip(TABLENAMES, [2, 37, 22]): # start_at corresponds to sheet indexes at which it switches to new dataset
    initialise_summary(
        data = DATA_LEEDS_BIKES[dataset]['all'], 
        lookup = VARIABLE_LOOKUP, 
        dataset_name = dataset, 
        key = 'all', 
        summary = SUMMARY_BIKES[dataset], 
        labels = LABELS[dataset], 
        plotting = PLOTTING[dataset], 
        fivenum = FIVENUM[dataset], 
        start_at=start_at)

In [ ]:
# time mapping
SUMMARY_BIKES['accidents']['all'][11]['Map'] = {i: f"{i}-{i+1}" for i in range(24)}
SUMMARY_BIKES['accidents']['all'][11]['Map'][-1] = 'N/A'

# date mapping
months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
SUMMARY_BIKES['accidents']['all'][9]['Map'] = {i+1: months[i] for i in range(12)}

# no. casualties and vehicles
SUMMARY_BIKES['accidents']['all'][7]['Map'] = {i: i for i in np.unique(DATA_LEEDS['accidents']['Number_of_Vehicles'])}
SUMMARY_BIKES['accidents']['all'][8]['Map'] = {i: i for i in np.unique(DATA_LEEDS['accidents']['Number_of_Casualties'])}

# fixed bug for windows file naming convention (no question marks allowed)
SUMMARY_BIKES['vehicles']['all'][12]['Name'] = 'Was_Vehicle_Left_Hand_Drive'

In [ ]:
for dataset in TABLENAMES:
    compute_numerical_summary(SUMMARY_BIKES[dataset]['all'], DATA_LEEDS_BIKES[dataset]['all'])

In [ ]:
for dataset in TABLENAMES:
    save_numerical_report(SUMMARY_BIKES[dataset]['all'], path=PATH['reports']['bikes'] + 'all/' + PATH[dataset] + 'numerical_summary/', filename=dataset, save_to='csv')
    save_numerical_report(SUMMARY_BIKES[dataset]['all'], path=PATH['reports']['bikes'] + 'all/' + PATH[dataset] + 'numerical_summary/', filename=dataset, save_to='json')

In [ ]:
%%capture 
for dataset in TABLENAMES:
    save_all_single_variable_analysis(SUMMARY_BIKES[dataset]['all'], path=PATH['reports']['bikes'] + 'all/' + PATH[dataset] + 'single_variable_analysis/', missing_values=False)

#### b. Analysis of Bikers and Motor Vehicle Driver
---

In [ ]:
for i in TABLENAMES:
    SUMMARY_BIKES[dataset]['bikers'] = {}
    SUMMARY_BIKES[dataset]['motorised'] = {}

In [ ]:
for dataset, start_at in zip(TABLENAMES[1:], [37, 22]): # start_at corresponds to sheet indexes at which it switches to new dataset
    initialise_summary(
        data = DATA_LEEDS_BIKES[dataset]['all'], 
        lookup = VARIABLE_LOOKUP, 
        dataset_name = dataset, 
        key = 'bikers', 
        summary = SUMMARY_BIKES[dataset], 
        labels = LABELS[dataset], 
        plotting = PLOTTING[dataset], 
        fivenum = FIVENUM[dataset], 
        start_at=start_at)
    initialise_summary(
        data = DATA_LEEDS_BIKES[dataset]['all'], 
        lookup = VARIABLE_LOOKUP, 
        dataset_name = dataset, 
        key = 'motorised', 
        summary = SUMMARY_BIKES[dataset], 
        labels = LABELS[dataset], 
        plotting = PLOTTING[dataset], 
        fivenum = FIVENUM[dataset], 
        start_at=start_at)

In [ ]:
for dataset in TABLENAMES:
    if dataset != 'accidents':
        for sub_dataset in ['bikers', 'motorised']:
            compute_numerical_summary(SUMMARY_BIKES[dataset][sub_dataset], DATA_LEEDS_BIKES[dataset][sub_dataset])

In [ ]:
for dataset in TABLENAMES:
    if dataset != 'accidents':
        for sub_dataset in ['bikers', 'motorised']:
            save_numerical_report(SUMMARY_BIKES[dataset][sub_dataset], path=PATH['reports']['bikes'] + f'{sub_dataset}/' + PATH[dataset] + 'numerical_summary/', filename=dataset, save_to='csv')
            save_numerical_report(SUMMARY_BIKES[dataset][sub_dataset], path=PATH['reports']['bikes'] + f'{sub_dataset}/' + PATH[dataset] + 'numerical_summary/', filename=dataset, save_to='json')

In [ ]:
%%capture 
for dataset in TABLENAMES:
    if dataset != 'accidents':
        for sub_dataset in ['bikers', 'motorised']:
            save_all_single_variable_analysis(SUMMARY_BIKES[dataset][sub_dataset], path=PATH['reports']['bikes'] + f'{sub_dataset}/' + PATH[dataset] + 'single_variable_analysis/', missing_values=False)

#### c. Findings
---

#### Most Common Bike Accident
94.2% of the bicycle accidents occur with cars or taxis, and in almost all of those cases the bicyclist gets injured. In contrast,only four drivers of motorised vehicles are being injured in all recorded bike accidents. By looking at where these accidentsoccur, it can be seen that in most cases the accidents are at junctions, where the cyclist is going straight and the car is turning,and the collision is front to front. This suggests that in most bicycle accident scenarios the car driver turns into the bicycliston accident. This recurring pattern of accident participants, manoeuvres, and location of accidents can and should be used toactively prevent these patterns of accident scenarios from happening. This could ie. be achieved through bike-specific trafficlights, longer delay periods after red lights or similar measures.

In [ ]:
bike_accident_junction_location = barplot(SUMMARY_BIKES['accidents']['all'][18], keep_missing_values=False)
vehicle_manouvre_bike = barplot(SUMMARY_BIKES['vehicles']['bikers'][4], keep_missing_values=False)
vehicle_manouvre_motorised_vehicle = barplot(SUMMARY_BIKES['vehicles']['motorised'][4], keep_missing_values=False)
first_point_of_impact_bike = barplot(SUMMARY_BIKES['vehicles']['bikers'][11], keep_missing_values=False)
first_point_of_impact_motorised_vehicle = barplot(SUMMARY_BIKES['vehicles']['motorised'][11], keep_missing_values=False)

### On the Way to Work
The data gives various arguments for the fact that most bike accidents happen during the rush hour times from 6-9 and 15-19.Firstly, most of the participants involved in bike accidents stated to be on the way to or from their workplace. This can also beseen in the distribution of accidents in different times of the day. The road collision numbers peak during the rush hours. Thedistribution of bike accidents during the week and the year further strengthen the above argument. On days of the weekendsthere occur noticeably less accidents when compared to regular week days. In the distribution of bike accidents per month, itis visible that the number of bike collisions drop in the month of August, which is the month of national summer holiday inthe UK. Therefore less people commute to or from work, and thus the number of bike accidents d



In [ ]:
bike_accidents_purpose = barplot(SUMMARY_BIKES['vehicles']['all'][13],  keep_missing_values=False) # purpose
bike_accidents_time = barplot(SUMMARY_BIKES['accidents']['all'][11], keep_missing_values=False) # time of the day
bike_accidents_day = barplot(SUMMARY_BIKES['accidents']['all'][10], keep_missing_values=False) # day of the week
bike_accidents_month = barplot(SUMMARY_BIKES['accidents']['all'][9], keep_missing_values=False) # month of the year

### Hazardous, Big Roads
Most of the accidents happen on streets classified as class A, which in the UK refers to main carriageways with high speedand little bends. The three most hazardous roads are the roads with road number 660 (Woodhouse Lane or Otley Road),65 (Kirkstall Road) and 61 (Regent Street). Together over 15% of all accidents involving bikes occur on these streets.Naturally, these roads have more accidents happening since they are more trafficked throughout the year than less centralroads. However, the high percentage of bike accidents happening at the three mentioned road, gives reason to believe that theroads are poorly secured for safe bike-travel and should therefore be a focus of political measurements trying to reduce thenumber of bike accidents in Lee

In [ ]:
road_class = barplot(SUMMARY_BIKES['accidents']['all'][14], keep_missing_values=False)
road_number = barplot(SUMMARY_BIKES['accidents']['all'][15], keep_missing_values=False)

### 3. Associations
---
We first associate all variabels that make sense to associate, in order to find the factor influencing the severity and frequency of accidents involving bikes the most. 

In [ ]:
SEVERITY_BIKES = {}
for dataset in TABLENAMES:
    SEVERITY_BIKES[dataset] = link_to_accidents(DATA_LEEDS_BIKES[dataset]['all'], focus='Accident_Severity')

In [ ]:
%%capture
for dataset in TABLENAMES:
    save_all_categorical_scatters(data = DATA_LEEDS_BIKES[dataset]['all'], summary = SUMMARY_BIKES[dataset]['all'], severity=SEVERITY_BIKES[dataset], path=PATH['reports']['bikes'] + 'all/' + PATH[dataset] + 'associations/')
    save_all_categorical_associations(data = DATA_LEEDS_BIKES[dataset]['all'], severity_summary = SUMMARY['accidents'], dataset_name = dataset, summary=SUMMARY_BIKES[dataset]['all'], severity= SEVERITY_BIKES[dataset], path=PATH['reports']['bikes'] + 'all/' + PATH[dataset] + 'associations/')

### 4. Spatial Visualisation
---
Finally, we would like to have a nice geospatial representation of where our bike accidents happened. With the following cell we generate maps that color-code the plotted accidents depending on different variables using the same code as in section `TASK 3`. All maps are exported in `html` format and can be viewed when opening in a browser of choice. The maps are saved in the directory `../reports/bikes/maps`.

**REMARK**: Note that the execution of the below cell may take some time, since several maps (each ~3MB) need to be rendered.

In [ ]:
%%capture
for boolean in [True, False]:
    _map = map_accidents(DATA_LEEDS_BIKES['accidents']['all'], SUMMARY_BIKES['accidents']['all'], centroid=[53.8008, -1.5491], heat_map=True, marker_cluster=boolean, focus='Accident_Severity')
    save_map(_map, PATH['reports']['bikes'] + 'maps/', filename=f"Accident_Severity_Map_(marker={boolean})")

### Bikes Accidents in Leeds (Focus on Accident Severity)
---

In [ ]:
bike_severity = map_accidents(DATA_LEEDS_BIKES['accidents']['all'], SUMMARY_BIKES['accidents']['all'], centroid=[53.8008, -1.5491], marker_cluster=True, focus='Accident_Severity')
bike_severity